Import packages

In [75]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Open the data file for reading

In [76]:
#save file in Pandas DataFrame
rawData = pd.read_csv('PGA Tour Raw Data.csv')
rawData.head()

,Player_initial_last,tournament id,player id,hole_par,strokes,hole_DKP,hole_FDP,hole_SDP,streak_DKP,streak_FDP,streak_SDP,n_rounds,made_cut,pos,finish_DKP,finish_FDP,finish_SDP,total_DKP,total_FDP,total_SDP,player,Unnamed: 2,Unnamed: 3,Unnamed: 4,tournament name,course,date,purse,season,no_cut,Finish,sg_putt,sg_arg,sg_app,sg_ott,sg_t2g,sg_total
0,A. Ancer,401353224,9261,288,289,60.0,51.1,56,3,7.6,3,4,1,32.0,2,1,0,65.0,59.7,59,Abraham Ancer,NaN,NaN,NaN,The Memorial Tournament pres. by Nationwide,"Muirfield Village Golf Club - Dublin, OH",6/5/22,12.0,2022,0,T32,0.20,-0.13,-0.08,0.86,0.65,0.85
1,A. Hadwin,401353224,5548,288,286,72.5,61.5,61,8,13.0,3,4,1,18.0,5,4,2,85.5,78.5,66,Adam Hadwin,NaN,NaN,NaN,The Memorial Tournament pres. by Nationwide,"Muirfield Village Golf Club - Dublin, OH",6/5/22,12.0,2022,0,T18,0.36,0.75,0.31,0.18,1.24,1.60
2,A. Lahiri,401353224,4989,144,147,21.5,17.4,27,0,0.0,0,2,0,NaN,0,0,0,21.5,17.4,27,Anirban Lahiri,NaN,NaN,NaN,The Memorial Tournament pres. by Nationwide,"Muirfield Village Golf Club - Dublin, OH",6/5/22,12.0,2022,0,CUT,-0.56,0.74,-1.09,0.37,0.02,-0.54
3,A. Long,401353224,6015,144,151,20.5,13.6,17,0,0.4,0,2,0,NaN,0,0,0,20.5,14.0,17,Adam Long,NaN,NaN,NaN,The Memorial Tournament pres. by Nationwide,"Muirfield Village Golf Club - Dublin, OH",6/5/22,12.0,2022,0,CUT,-1.46,-1.86,-0.02,0.80,-1.08,-2.54
4,A. Noren,401353224,3832,144,148,23.5,18.1,23,0,1.2,0,2,0,NaN,0,0,0,23.5,19.3,23,Alexander Noren,NaN,NaN,NaN,The Memorial Tournament pres. by Nationwide,"Muirfield Village Golf Club - Dublin, OH",6/5/22,12.0,2022,0,CUT,0.53,-0.36,-1.39,0.19,-1.56,-1.04


clean the data to eliminate unnecessary columns

In [77]:
#dropping any rows/columns with missing data
null_columns = rawData.isnull().any()

# Print columns with null value
print(null_columns)

Player_initial_last    False
tournament id          False
player id              False
hole_par               False
strokes                False
hole_DKP               False
hole_FDP               False
hole_SDP               False
streak_DKP             False
streak_FDP             False
streak_SDP             False
n_rounds               False
made_cut               False
pos                     True
finish_DKP             False
finish_FDP             False
finish_SDP             False
total_DKP              False
total_FDP              False
total_SDP              False
player                 False
Unnamed: 2              True
Unnamed: 3              True
Unnamed: 4              True
tournament name        False
course                 False
date                   False
purse                  False
season                 False
no_cut                 False
Finish                  True
sg_putt                 True
sg_arg                  True
sg_app                  True
sg_ott        

In [78]:
#create a new dataframe with only the columns that are important for analysis
columns_to_drop = ["Player_initial_last", "hole_DKP", "hole_FDP", "hole_SDP", "streak_DKP",
                    "streak_FDP", "streak_SDP", "made_cut", "pos", "finish_DKP", "finish_FDP",
                    "finish_SDP", "total_DKP", "total_FDP", "total_SDP", "Unnamed: 2", "Unnamed: 3",
                    "Unnamed: 4", "purse", "no_cut"]

cleanData = rawData.drop(columns=columns_to_drop)

In [79]:
#Clean up any rows with a "n_rounds" value not equal to 4, to keep data points consistent
cleanData = cleanData[cleanData.n_rounds == 4]

# Clean up any rows with a 'sg_total' value of NA
cleanData = cleanData.dropna(subset=['sg_total'])

#Determine and save the total average strokes for each tournament, and the average score to par
scoreToPar = cleanData['strokes']-cleanData['hole_par']
cleanData['score_to_par'] = scoreToPar

# Print the first 5 rows of the new data
print(cleanData.head())

   tournament id  player id  hole_par  strokes  n_rounds         player                              tournament name                                    course    date  season Finish  sg_putt  sg_arg  sg_app  sg_ott  sg_t2g  sg_total  score_to_par
0      401353224       9261       288      289         4  Abraham Ancer  The Memorial Tournament pres. by Nationwide  Muirfield Village Golf Club - Dublin, OH  6/5/22    2022    T32     0.20   -0.13   -0.08    0.86    0.65      0.85             1
1      401353224       5548       288      286         4    Adam Hadwin  The Memorial Tournament pres. by Nationwide  Muirfield Village Golf Club - Dublin, OH  6/5/22    2022    T18     0.36    0.75    0.31    0.18    1.24      1.60            -2
6      401353224      10906       288      287         4      Aaron Rai  The Memorial Tournament pres. by Nationwide  Muirfield Village Golf Club - Dublin, OH  6/5/22    2022    T26     2.05    0.74   -1.32   -0.12   -0.70      1.35            -1
7      40135

Now, we will separate the data by region into separate dataframes.
I have decided to select the top 8 yearly tournaments from each region to analyze.
The regions are Northeast, Southeast, and West (There is not much going on in the northwest, so I combined north and south)

In [80]:
valuesNE = ['3M Open', 'John Deere Classic', 'BMW Championship', 'Rocket Mortgage Classic',
            'RBC Canadian Open', 'Barbasol Championship', 'Travelers Championship']

# Select rows where 'tournament name' is in the specified values
northEastRegion = cleanData[cleanData['tournament name'].isin(valuesNE)]

# pd.set_option('display.max_columns', None)
# pd.set_option('display.expand_frame_repr', False)
# pd.set_option('max_colwidth', None)
# pd.set_option('display.max_rows', None)


# # Print the entire DataFrame
# print(northEastRegion)

In [81]:
#Now, do the same for the other regions

#SouthEast
valuesSE = ['The Honda Classic', 'Valspar Championship', 'Arnold Palmer Invitational Pres. by Mastercard', 'RBC Heritage',
            'Masters Tournament', 'Sanderson Farms Championship', 'Tour Championship']
southEastRegion = cleanData[cleanData['tournament name'].isin(valuesSE)]

#West
valuesWest = ['Valero Texas Open', 'Shriners Hospitals for Children Open', 'Waste Management Phoenix Open',
            'AT&T Byron Nelson', 'The American Express', 'Farmers Insurance Open',
            'The Genesis Invitational', 'Charles Schwab Challenge']
westRegion = cleanData[cleanData['tournament name'].isin(valuesWest)]

Now, ensure the data was successfully copied into these new dataframes

In [82]:
# Check which columns have null values
null_columns = northEastRegion.isnull().any()
column_types = northEastRegion.dtypes

# Print columns with null value
print(null_columns)
print(column_types)

tournament id      False
player id          False
hole_par           False
strokes            False
n_rounds           False
player             False
tournament name    False
course             False
date               False
season             False
Finish             False
sg_putt            False
sg_arg             False
sg_app             False
sg_ott             False
sg_t2g             False
sg_total           False
score_to_par       False
dtype: bool
tournament id        int64
player id            int64
hole_par             int64
strokes              int64
n_rounds             int64
player              object
tournament name     object
course              object
date                object
season               int64
Finish              object
sg_putt            float64
sg_arg             float64
sg_app             float64
sg_ott             float64
sg_t2g             float64
sg_total           float64
score_to_par         int64
dtype: object


In [83]:
# Check which columns have null values
null_columns = southEastRegion.isnull().any()
column_types = southEastRegion.dtypes

# Print columns with null value
print(null_columns)
print(column_types)

tournament id      False
player id          False
hole_par           False
strokes            False
n_rounds           False
player             False
tournament name    False
course             False
date               False
season             False
Finish             False
sg_putt            False
sg_arg             False
sg_app             False
sg_ott             False
sg_t2g             False
sg_total           False
score_to_par       False
dtype: bool
tournament id        int64
player id            int64
hole_par             int64
strokes              int64
n_rounds             int64
player              object
tournament name     object
course              object
date                object
season               int64
Finish              object
sg_putt            float64
sg_arg             float64
sg_app             float64
sg_ott             float64
sg_t2g             float64
sg_total           float64
score_to_par         int64
dtype: object


In [84]:
# Check which columns have null values
null_columns = westRegion.isnull().any()
column_types = westRegion.dtypes

# Print columns with null value
print(null_columns)
print(column_types)

tournament id      False
player id          False
hole_par           False
strokes            False
n_rounds           False
player             False
tournament name    False
course             False
date               False
season             False
Finish             False
sg_putt            False
sg_arg             False
sg_app             False
sg_ott             False
sg_t2g             False
sg_total           False
score_to_par       False
dtype: bool
tournament id        int64
player id            int64
hole_par             int64
strokes              int64
n_rounds             int64
player              object
tournament name     object
course              object
date                object
season               int64
Finish              object
sg_putt            float64
sg_arg             float64
sg_app             float64
sg_ott             float64
sg_t2g             float64
sg_total           float64
score_to_par         int64
dtype: object


Now that the data is cleaned and sorted into the different regions, it is time to begin analysing the data 

In [85]:
# Create a dictionary to store each tournament name and the average strokes for that tournament
# Get the unique tournament names
tournament_names = northEastRegion['tournament name'].unique()

# Initialize an empty dictionary to store the averages
strokeAveragesNE = {}

for name in tournament_names:

    selected_rows = northEastRegion[northEastRegion['tournament name'] == name]
    
    # Calculate the mean of 'strokes' for the selected rows
    stroke_mean = np.mean(selected_rows['strokes'])
    
    # Add the tournament name and average to the dictionary
    strokeAveragesNE[name] = stroke_mean

# Print the dictionary
print(strokeAveragesNE)

{'BMW Championship': 279.3667481662592, '3M Open': 274.9389671361502, 'Barbasol Championship': 274.1766666666667, 'John Deere Classic': 273.9139240506329, 'Rocket Mortgage Classic': 277.2258064516129, 'Travelers Championship': 274.2636579572447, 'RBC Canadian Open': 264.9797687861272}


In [86]:
# Now, do the same for the other regions
#SouthEast
tournament_names = southEastRegion['tournament name'].unique()

strokeAveragesSE = {}

for name in tournament_names:

    selected_rows = southEastRegion[southEastRegion['tournament name'] == name]
    
    stroke_mean = np.mean(selected_rows['strokes'])
    
    strokeAveragesSE[name] = stroke_mean

print(strokeAveragesSE)

{'RBC Heritage': 279.35346358792185, 'Masters Tournament': 292.2857142857143, 'Valspar Championship': 282.8678304239402, 'Arnold Palmer Invitational Pres. by Mastercard': 288.3407258064516, 'The Honda Classic': 281.75735294117646, 'Sanderson Farms Championship': 280.49187935034803, 'Tour Championship': 278.13106796116506}
